In [ ]:
"""""""""
Written by Mengzhan Liufu and Sameera Shridhar at Yu Lab, the University of Chicago, November 2021
"""""""""
import Ipynb_importer

In [ ]:
import TrodesReader
from collections import deque
import numpy as np
import math
from scipy import signal
import matplotlib.pyplot as plt

In [ ]:
def bandpass_filter(filter_name, flattened_array, sampling_freq, order, lowcut, highcut):
    """
    Return a dictionary of filtered lfp data

    :param filter_name: name of the filter you want to use
    :type filter_name: string
    :param flattened_array: array of the raw lfp data
    :type flattened_array: np array
    :param sampling_freq: frequency lfp data was sampled at
    :type sampling_freq: int
    :param order: order of filter
    :type order: int
    :param lowcut: lower border of frequencies allowed to pass
    :type lowcut: int
    :param highcut: upper border of frequencies allowed to pass
    :type highcut: int

    :return: filtered lfp data
    :rtype: np array
    """
    if filter_name == 'elliptical':
        sos = signal.ellip(order, 0.01, 120, [lowcut, highcut], btype='bp', output='sos', fs=sampling_freq)
    if filter_name == 'butterworth':
        sos = signal.butter(order, [lowcut, highcut], 'bp', fs=sampling_freq, output='sos')
    if filter_name == 'cheby1':
        sos = signal.cheby1(order, 1, [lowcut, highcut], 'bp', fs=sampling_freq, output='sos')
    if filter_name == 'cheby2':
        sos = signal.cheby2(order, 15, [lowcut, highcut], 'bp', fs=sampling_freq, output='sos')

    y = signal.sosfiltfilt(sos, flattened_array)

    return y

def calculate_rms(buffer):
    """
    return the root mean-squared of a given array
    :param buffer: any array or list of number

    :return: the root mean-squared value of the array as a proxy for its power
    :rtype: float
    """
    square_summed = 0
    for k in buffer:
        square_summed += (k**2)

    return math.sqrt(square_summed/len(buffer))


def detection_with_rms(buffer, low_cut, high_cut, threshold):
    """
    :param buffer: the buffer of lfp data at current iteration
    :param low_cut: the lower bound of the frequency band of interest
    :param high_cut: the upper bound of the frequency band of interest
    :param threshold: the threshold of power for making decision/judgement

    :return: whether there is activity in freq range [low_cut, high_cut] or not
    :rtype: boolean
    """
    filtered_buffer = bandpass_filter('butterworth', buffer, lfp_sampling_rate, 0, low_cut, high_cut)
    current_rms = calculate_rms(filtered_buffer)
    if current_rms >= threshold:
        return True
    else:
        return False

In [ ]:
lfp_sampling_rate = 1250
lfp_sampling_period = (1/1250)*(10**9)
fNQ = 600

data_path = r'C:\Users\John LauFoo\Box\Jhan\ClosedLoopControl Project\DATA\20211117_testing_5min\20211117_testing_5min.LFP\20211117_testing_5min.LFP_nt28ch1.dat'
timestamp_path = r'C:\Users\John LauFoo\Box\Jhan\ClosedLoopControl Project\DATA\20211117_testing_5min\20211117_testing_5min.LFP\20211117_testing_5min.timestamps.dat'

In [ ]:
data = TrodesReader.readTrodesExtractedDataFile(data_path)
timestamps = TrodesReader.readTrodesExtractedDataFile(timestamp_path)

lfp_data = []
for i in data['data']:
    lfp_data.append(i[0])

In [ ]:
ripple_range_filtered_data = bandpass_filter('butterworth', lfp_data, lfp_sampling_rate, 1, 140, 225)
sharp_wave_range_filtered_data = bandpass_filter('butterworth', lfp_data, lfp_sampling_rate, 1, 5, 15)
gamma_range_filtered_data = bandpass_filter('butterworth', lfp_data, lfp_sampling_rate, 1, 20, 40)

In [ ]:
ripple_rms_history = []
sharp_wave_rms_history = []
gamma_rms_history = []
for i in range(2499, len(ripple_range_filtered_data)):
    ripple_buffer = ripple_range_filtered_data[i-2499:i]
    ripple_rms_history.append(calculate_rms(ripple_buffer))
    sharp_wave_buffer = sharp_wave_range_filtered_data[i-2499:i]
    sharp_wave_rms_history.append(calculate_rms(sharp_wave_buffer))
    gamma_range_buffer = gamma_range_filtered_data[i-2499:i]
    gamma_rms_history.append(calculate_rms(gamma_range_buffer))

In [ ]:
ripple_avg_rms = np.mean(ripple_rms_history)
sharp_wave_avg_rms = np.mean(sharp_wave_rms_history)
gamma_avg_rms = np.mean(gamma_rms_history)

ripple_std_rms = np.std(ripple_rms_history)
sharp_wave_std_rms = np.std(sharp_wave_rms_history)
gamma_std_rms = np.std(gamma_rms_history)

ripple_rms_threshold = ripple_avg_rms + 4*ripple_std_rms
sharp_wave_rms_threshold = sharp_wave_avg_rms + 4*sharp_wave_std_rms
gamma_rms_threshold = gamma_avg_rms + 4*gamma_std_rms

In [ ]:
# Run the detection_with_rms() function independently on raw data

ripple_decision_list = [False, False, False]
sw_decision_list = [False, False, False]
gamma_decision_list = [False, False, False]

ripple_stimulation_list = []
sw_stimulation_list = []
gamma_stimulation_list = []

for i in range(2499, len(ripple_range_filtered_data)):
    buffer = lfp_data[i-2499:i]
    ripple_decision = detection_with_rms(buffer,140,225,ripple_rms_threshold)
    sw_decision = detection_with_rms(buffer,5,15,sharp_wave_rms_threshold)
    gamma_decision = detection_with_rms(buffer,20,40,gamma_rms_threshold)
    
    ripple_decision_list.append(ripple_decision)
    sw_decision_list.append(sw_decision)
    gamma_decision_list.append(gamma_decision)
    
    ripple_stimulation = False
    sw_stimulation = False
    gamma_stimulation = False
    for m in range(len(ripple_decision_list)-2, len(ripple_decision_list)):
        ripple_stimulation = ripple_decision_list[m]
        ripple_stimulation_list.append(ripple_stimulation)
        sw_stimulation = sw_decision_list[m]
        sw_stimulation_list.append(sw_stimulation)
        gamma_stimulation = gamma_decision_list[m]
        gamma_stimulation_list.append(gamma_stimulation)

In [ ]:
fig2 = plt.figure(figsize=(30,10))
plt.style.use('seaborn-white')
ripple_stimulation_plot = fig2.add_subplot(3,2,1)
ripple_decision_plot = fig2.add_subplot(3,2,2)
sw_stimulation_plot = fig2.add_subplot(3,2,3)
sw_decision_plot = fig2.add_subplot(3,2,4)
gamma_stimulation_plot = fig2.add_subplot(3,2,5)
gamma_decision_plot = fig2.add_subplot(3,2,6)

ripple_stimulation_plot.plot(ripple_stimulation_list)
ripple_stimulation_plot.set_ylabel('Decision',fontsize=30)
ripple_decision_plot.plot(ripple_decision_list)

sw_stimulation_plot.plot(sw_stimulation_list)
sw_stimulation_plot.set_ylabel('Decision',fontsize=30)
sw_decision_plot.plot(sw_decision_list)

gamma_stimulation_plot.plot(gamma_stimulation_list)
gamma_stimulation_plot.set_ylabel('Decision',fontsize=30)
gamma_stimulation_plot.set_xlabel('Sample # (Stimulation)',fontsize=30)
gamma_decision_plot.plot(gamma_decision_list)
gamma_decision_plot.set_xlabel('Sample # (Decision)',fontsize=30)

plt.suptitle('Detection Decision and Stimulation at Each Sample Point',fontsize=30)
plt.show()

In [ ]:
fig1 = plt.figure(figsize=(90,60))

ripple_plot = fig1.add_subplot(3,1,1)
raw_data_plot = fig1.add_subplot(3,1,2)
ripple_range_filtered_plot = fig1.add_subplot(3,1,3)

ripple_plot.plot(ripple_rms_history,'r')
ripple_plot.set_ylabel('RMS of LFP',fontsize=70)
ripple_plot.set_xlabel('Sample #',fontsize=100)
ripple_plot.grid(True)
ripple_plot.set_title('Ripple Range (140~225 Hz)',fontsize=70)

raw_data_plot.plot(lfp_data,'k')
raw_data_plot.set_ylabel('LFP Raw Data',fontsize=70)
raw_data_plot.set_xlabel('Sample #',fontsize=100)
raw_data_plot.grid(True)

ripple_range_filtered_plot.plot(ripple_range_filtered_data,'b')
ripple_range_filtered_plot.set_ylabel('Ripple Range Filtered',fontsize=70)
ripple_range_filtered_plot.set_xlabel('Sample #',fontsize=100)
ripple_range_filtered_plot.grid(True)

plt.style.use('seaborn-white')
plt.show()

In [ ]:
plt.figure(figsize=(60,30))
plt.plot(lfp_data,color='k',alpha=0.5)
plt.plot(ripple_range_filtered_data,color='b',alpha=0.5)
plt.plot(ripple_rms_history,color='r',alpha=0.5)
plt.style.use('seaborn-white')
plt.show()

In [ ]:
fig4 = plt.figure(figsize=(40,20))
grid = plt.GridSpec(6,1,wspace=0.5, hspace=0)
ax1 = plt.subplot(grid[0:4,0:1])
ax2 = plt.subplot(grid[4:6,0:1])

ax1.plot(lfp_data,color='k',alpha=0.5)
ax1.plot(ripple_range_filtered_data,color='b',alpha=0.5)
ax1.grid(True)
ax1.set_ylabel('LFP',fontsize=40)
ax2.plot(ripple_rms_history,color='r')
ax2.grid(True)
ax2.set_xlabel('Sample #',fontsize=40)
ax2.set_ylabel('RMS',fontsize=40)

plt.style.use('seaborn-white')
plt.suptitle('Ripple Range (Whole Recording)',fontsize=50)
plt.show()

In [ ]:
fig1 = plt.figure(figsize=(90,60))

sw_plot = fig1.add_subplot(3,1,1)
raw_data_plot = fig1.add_subplot(3,1,2)
sw_range_filtered_plot = fig1.add_subplot(3,1,3)

sw_plot.plot(sharp_wave_rms_history,'r')
sw_plot.set_ylabel('RMS of LFP',fontsize=70)
sw_plot.set_xlabel('Sample #',fontsize=100)
sw_plot.grid(True)
sw_plot.set_title('Sharp-Wave Range (140~225 Hz)',fontsize=70)

raw_data_plot.plot(lfp_data,'k')
raw_data_plot.set_ylabel('LFP Raw Data',fontsize=70)
raw_data_plot.set_xlabel('Sample #',fontsize=100)
raw_data_plot.grid(True)

sw_range_filtered_plot.plot(ripple_range_filtered_data,'b')
sw_range_filtered_plot.set_ylabel('Sharp-Wave Range Filtered',fontsize=70)
sw_range_filtered_plot.set_xlabel('Sample #',fontsize=100)
sw_range_filtered_plot.grid(True)

plt.style.use('seaborn-white')
plt.show()